In [1]:
from config import *
from pyBKT.models import Model
from main_package.utils import data_path_to_abs_path
import numpy as np
import pandas as pd
np.seterr(divide='ignore', invalid='ignore') #pyBKT raises some warnings that I should ignore

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

Applying BKT to piech's data

In [2]:
defaults = {'skill_name': 'skill_id'}
model = Model(seed = 42, num_fits = 2)

In [4]:
model.fit(data_path = data_path_to_abs_path("piech/train_df_format.csv"), defaults=defaults)
print(model.params())

                        value
skill param   class          
82    prior   default 0.77088
      learns  default 0.08808
      guesses default 0.09506
      slips   default 0.20276
      forgets default 0.00000
...                       ...
28    prior   default 0.14712
      learns  default 1.00000
      guesses default 0.50000
      slips   default 0.50000
      forgets default 0.00000

[615 rows x 1 columns]


In [ ]:
#model.predict(data_path = data_path_to_abs_path("piech/test_df_format"))

In [5]:
auc = model.evaluate(data_path = data_path_to_abs_path("piech/test_df_format.csv"), metric='auc')
print(f"auc: {auc}")
acc = model.evaluate(data_path = data_path_to_abs_path("piech/test_df_format.csv"), metric='accuracy')
print(f"accuracy: {acc}")

auc: 0.7547118541968068
accuracy: 0.7493108963451984


In [7]:
model2 = Model(seed = 42, num_fits = 2)
model2.fit(data_path = data_path_to_abs_path("piech/train_df_format.csv"), defaults=defaults, forgets=True)
print(model2.params())

                        value
skill param   class          
82    prior   default 0.78255
      learns  default 0.11263
      guesses default 0.05109
      slips   default 0.20140
      forgets default 0.00460
...                       ...
28    prior   default 0.09779
      learns  default 1.00000
      guesses default 0.50000
      slips   default 0.50000
      forgets default 0.00000

[615 rows x 1 columns]


In [8]:
auc = model2.evaluate(data_path = data_path_to_abs_path("piech/test_df_format.csv"), metric='auc')
print(f"auc: {auc}")
acc = model2.evaluate(data_path = data_path_to_abs_path("piech/test_df_format.csv"), metric='accuracy')
print(f"accuracy: {acc}")

auc: 0.8261624759154647
accuracy: 0.7823283196079766


# Trying custom params

In [14]:
def provide_values_for_custom_params(p_L0, p_G, p_S, p_T, p_F):
    return {
        "prior": p_L0,
        "learns": np.array([p_T]),
        "guesses": np.array([p_G]),
        "slips": np.array([p_S]),
        "forgets": np.array([p_F]),
    }

df_sample = pd.read_csv(data_path_to_abs_path("piech/sample_df_format"))
df_sample['skill_id'].max()

123

In [15]:
custom_params = {
    str(i): provide_values_for_custom_params(0.5504063365919529, 0.3, 0.0999999975569658, 0.06302321901515896, 0) for i in range(124)
}
model.coef_ = custom_params

In [17]:
df_piech = pd.read_csv(data_path_to_abs_path("piech/sample_df_format"))

In [18]:
model.fit(data = df_piech.head(100), defaults=defaults, multilearn=False, fixed=True)


In [10]:
def pretty(d, indent=0):
   for key, value in d.items():
      print(' ' * indent + str(key))
      if isinstance(value, dict):
         pretty(value, indent+1)
      else:
         print(' ' * (indent+1) + str(value))

In [19]:
#print(model.coef_)
#pretty(model.fit_model)

print(model.params())

KeyError: 'resource_names'

In [20]:
model.predict(data_path = data_path_to_abs_path("piech/sample_df_format"))

,user_id,skill_id,correct,correct_predictions,state_predictions
0,0,51,0,0.63024,0.55041
1,0,51,1,0.42150,0.20250
2,0,51,1,0.58089,0.46815
3,0,51,1,0.74558,0.74264
4,1,82,0,0.63024,0.55041
...,...,...,...,...,...
406,10,123,1,0.89996,0.99994
407,10,123,0,0.89999,0.99998
408,10,123,0,0.89992,0.99987
409,10,123,1,0.89948,0.99914


In [21]:
auc = model.evaluate(data = df_piech, metric = 'auc')